In [ ]:
!nvidia-smi

In [ ]:
import glob
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import argparse
import random
import os, sys
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torchvision.models as models

from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

sys.path.append('../') # import를 위해 경로추가
from Modules import Utility as U
from Modules import CustomDataset 
from Modules import SmartCrop

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, default=20)
parser.add_argument('--lr', type=float, default=0.001)
parser.add_argument('--batch_size', type=int, default=32)
parser.add_argument('--seed', type=int, default=999)
parser.add_argument('--img_size', type=int, default=512)
parser.add_argument('--device', default='cuda')
parser.add_argument('--data_path', type=str, default='../data/')

args = parser.parse_args('')
args

In [ ]:
U.seed_everything(args.seed)

In [ ]:
df = pd.read_csv(args.data_path+'train_repaired.csv')

In [ ]:
le = preprocessing.LabelEncoder()
df['artist'] = le.fit_transform(df['artist'].values)
df.head()

In [ ]:
train_df, val_df, _, _ = train_test_split(df, df['artist'].values, test_size=0.2, random_state=args.seed)

In [ ]:
train_img_paths, train_labels = U.get_data(train_df)
val_img_paths, val_labels = U.get_data(val_df)

In [ ]:
# random flip
train_transform = A.Compose([
                            SmartCrop(),
                            A.Resize(args.img_size,args.img_size),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            SmartCrop(),
                            A.Resize(args.img_size,args.img_size),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

In [ ]:
train_dataset = CustomDataset(args.data_path, train_img_paths, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = args.batch_size, shuffle=True, num_workers=0)

val_dataset = CustomDataset(args.data_path, val_img_paths, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0)

In [ ]:
target_model = models.efficientnet_v2_s(weights='DEFAULT', pretrained=True)

In [ ]:
for p in target_model.parameters():
    p.requires_grad = True
target_model.classifier[1] = torch.nn.Linear(in_features=1280, out_features=50, bias=True)

In [ ]:
def validation(model, criterion, test_loader, device):
    model.eval()
    
    model_preds = []
    true_labels = []
    
    val_loss = []
    
    with torch.no_grad():
        for item in tqdm(test_loader):
            img = item['image']
            label = item['label']
            h = item['height']
            w = item['width']
            img, label = img.float().to(device), label.to(device)
            model_pred = model(img)
            
            loss = criterion(model_pred, label)
            
            val_loss.append(loss.item())
            
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += label.detach().cpu().numpy().tolist()
        
    val_f1 = U.competition_metric(true_labels, model_preds)
    return np.mean(val_loss), val_f1

In [ ]:
def train(model, optimizer, train_loader, test_loader, scheduler, device):
    model.to(device)

    criterion = nn.CrossEntropyLoss().to(device)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, args.epochs+1):
        model.train()
        train_loss = []
        for item in tqdm(train_loader):
            img = item['image']
            label = item['label']
            h = item['height']
            w = item['width']
            img, label = img.float().to(device), label.to(device)
            optimizer.zero_grad()

            model_pred = model(img)
            
            loss = criterion(model_pred, label)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        tr_loss = np.mean(train_loss)
            
        val_loss, val_score = validation(model, criterion, test_loader, device)
            
        print(f'Epoch [{epoch}], Train Loss : [{tr_loss:.5f}] Val Loss : [{val_loss:.5f}] Val F1 Score : [{val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step()
            
        if best_score < val_score:
            best_model = model
            best_score = val_score
        
    return best_model

In [ ]:
model = target_model.to(args.device)
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = args.lr)
scheduler = None
# scheduler = torch.optim.StepLR(optimizer, step_size=15, gamma=0.1)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, args.device)

In [ ]:
torch.save({
    'model': infer_model.state_dict(),
    'optim': optimizer.state_dict()
}, '20epoch_efficient_v2_1mlp_concat_model.tar')

In [ ]:
test_df = pd.read_csv(args.data_path + 'test.csv')

In [ ]:
test_img_paths = U.get_data(test_df, infer=True)

In [ ]:
test_dataset = CustomDataset(args.data_path, test_img_paths, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    model_preds = []
    
    with torch.no_grad():
        for item in tqdm(iter(test_loader)):
            img = item['image'].float().to(device)
            h = item['height']
            w = item['width']
            
            model_pred = model(img)
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
    
    print('Done.')
    return model_preds

In [ ]:
preds = inference(infer_model, test_loader, args.device)

In [ ]:
preds = le.inverse_transform(preds) 

In [ ]:
submit = pd.read_csv(args.data_path + 'sample_submission.csv')

In [ ]:
submit['artist'] = preds
submit.head()

In [ ]:
submit.to_csv('./efficient_b3_2mlp_concat.csv', index=False)

In [ ]:
torch.cuda.memory_reserved()

In [ ]:
torch.cuda.empty_cache()